# Test Parallel ExIFFI

In [1]:
import sys
import numpy as np
import pandas as pd
#from tqdm import tqdm
from append_dir import append_dirname
append_dirname('ExIFFI')
from utils.utils import partition_data
from utils.feature_selection import *
#from plot import *
#from simulation_setup import *
from models import *
from models.Extended_IF import *
from models.Extended_DIFFI_parallel import *
from models.Extended_DIFFI_original import *
import math
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler
import time

import os
import pickle 
from scipy.io import loadmat
from glob import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

/tmp/ipykernel_59299/2777325278.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Utility Functions

In [2]:
path=os.getcwd()
path=os.path.dirname(path)
path_real=os.path.join(path,'data','real')
path_syn=os.path.join(path,'data','synthetic')
mat_files_real=glob(os.path.join(path_real, '*.mat'))
mat_file_names_real={os.path.basename(x).split('.')[0]:x for x in mat_files_real}
files_syn=glob(os.path.join(path_syn, '*.pkl'))
file_names_syn={os.path.basename(x).split('.')[0]:x for x in files_syn}
csv_files=glob(os.path.join(path, '*.csv'))
csv_file_names={os.path.basename(x).split('.')[0]:x for x in csv_files}

In [3]:
def load_data_syn(filename,path):
    file_to_read = open(os.path.join(path,filename), "rb")
    dict = pickle.load(file_to_read)
    data=[]
    for k in dict.keys():
        data.append(dict[k])

    return data

In [4]:
file_to_read = open(os.path.join(path_syn,'ball_6_dim.pkl'), "rb")
loaded_dictionary = pickle.load(file_to_read)
X_train=loaded_dictionary['X_train']
file_to_read = open(os.path.join(path_syn,'anomalies.pkl'), "rb")
loaded_dictionary = pickle.load(file_to_read)
X_xaxis,X_yaxis,X_bisect,X_bisect_3d,X_bisect_6d=loaded_dictionary['X_xaxis'],loaded_dictionary['X_yaxis'],loaded_dictionary['X_bisec'],loaded_dictionary['X_bisec_3d'],loaded_dictionary['X_bisec_6d']
file_to_read = open(os.path.join(path_syn,'toy_datasets.pkl'), "rb")
loaded_dictionary = pickle.load(file_to_read)
X_toy_2d,X_toy_3d,X_toy_4d,X_toy_6d,y = loaded_dictionary['X_toy_2d'],loaded_dictionary['X_toy_3d'],loaded_dictionary['X_toy_4d'],loaded_dictionary['X_toy_6d'],loaded_dictionary['y']

In [5]:
def load_data(filename):
    data=loadmat(mat_file_names_real[filename])
    X,y=data['X'],data['y']
    y=np.hstack(y)
    return X,y 

def load_data_csv(filename):
    data=pd.read_csv(csv_file_names[filename])
    if 'Unnamed: 0' in data.columns:
        data=data.drop(columns=['Unnamed: 0'])
    X=data[data.columns[data.columns!='Target']]
    y=data['Target']
    return X,y

In [7]:
def compute_imps(model,X_train,X_test,n_runs,name,pwd,dim,f=6):

    name='GFI_'+name

    #X_test=np.r_[X_train,X_test]

    imps=np.zeros(shape=(n_runs,X_train.shape[1]))
    for i in tqdm(range(n_runs)):
        model.fit(X_train)
        imps[i,:]=model.Global_importance(X_test,calculate=True,overwrite=False,depth_based=False)

    path = pwd + '/results/imp/imp_score_' + name + '.pkl'
    with open(path, 'wb') as fl:
        pickle.dump(imps,fl)

    return imps

In [8]:
def interpretation_plots(name,pwd):

    os.chdir('c:\\Users\\lemeda98\\Desktop\\PHD Information Engineering\\ExIFFI\\ExIFFI\\data')
    if name=='diabetes' or name=='moodify':
        X,y=csv_dataset(name,os.getcwd()+'\\')
    else:
        X,y=dataset(name,os.getcwd()+'\\')

    X,y=downsample(X,y)
    X_train,X_test=partition_data(X,y)
    scaler=StandardScaler()
    X_train=scaler.fit_transform(X_train)
    X_test=scaler.transform(X_test)
    y_train=np.zeros(X_train.shape[0])
    y_test=np.ones(X_test.shape[0])
    y=np.concatenate([y_train,y_test])
    X_test=np.r_[X_train,X_test]
    scaler2=StandardScaler()
    X=scaler2.fit_transform(X)
    EDIFFI=Extended_DIFFI_original(300,max_depth=100,subsample_size=256,plus=1)
    dim=X.shape[1]

    # No Split 

    imps,plt_data=compute_imps(EDIFFI,X,X,10,name,pwd,dim,f=6)

    path = pwd + '\\results\\davide\\Importance_Scores\\Imp Score\\imp_score_GFI_' + name + '.pkl'
    with open(path, 'rb') as fl:
        imps = pickle.load(fl)

## Load Data

## Wine Dataset

In [9]:
name='wine'
X,y=load_data(name)
X_train,X_test=partition_data(X,y)
X.shape,y.shape

((129, 13), (129,))

In [10]:
X_train.shape,X_test.shape

((119, 13), (10, 13))

In [22]:
EDIFFI=Extended_DIFFI_original(300,max_depth=100,subsample_size=256,plus=1,seed=126)

### Serial ExIFFI

In [7]:
from sklearn.preprocessing import StandardScaler
import time
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=np.zeros(X_train.shape[0])
y_test=np.ones(X_test.shape[0])
y=np.concatenate([y_train,y_test])
X_test=np.r_[X_train,X_test]
scaler2=StandardScaler()
X=scaler2.fit_transform(X)
EDIFFI=Extended_DIFFI_original(300,max_depth=100,subsample_size=256,plus=1)
dim=X.shape[1]
pwd=os.path.dirname(os.getcwd())
start=time.time()
imps,plt_data=compute_imps(EDIFFI,X,X,10,name,pwd,dim,f=6)
end=time.time()
print(f'Elapsed time: {end-start}')

### Parallel ExIFFI

In [12]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=np.zeros(X_train.shape[0])
y_test=np.ones(X_test.shape[0])
y=np.concatenate([y_train,y_test])
X_test=np.r_[X_train,X_test]
scaler2=StandardScaler()
X=scaler2.fit_transform(X)
EDIFFI=Extended_DIFFI_parallel(300,max_depth=100,subsample_size=256,plus=1)
EDIFFI.set_num_processes(8,8)

dim=X.shape[1]
pwd=os.path.dirname(os.getcwd())


start=time.time()
imps=compute_imps(EDIFFI,X,X,10,name,pwd,dim,f=6)
end=time.time()

print(f'Elapsed time: {end-start}')

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:35<00:00,  3.55s/it]

Elapsed time: 35.46605706214905


## Ionosphere Dataset

In [5]:
name='ionosphere'
X,y=load_data(name)
X_train,X_test=partition_data(X,y)
X.shape,y.shape

((351, 33), (351,))

### Serial ExIFFI

In [19]:
from sklearn.preprocessing import StandardScaler
import time
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=np.zeros(X_train.shape[0])
y_test=np.ones(X_test.shape[0])
y=np.concatenate([y_train,y_test])
X_test=np.r_[X_train,X_test]
scaler2=StandardScaler()
X=scaler2.fit_transform(X)
EDIFFI=Extended_DIFFI_original(300,max_depth=100,subsample_size=256,plus=1)
dim=X.shape[1]
pwd=os.path.dirname(os.getcwd())
start=time.time()
imps,plt_data=compute_imps(EDIFFI,X,X,10,name,pwd,dim,f=6)
end=time.time()
print(f'Elapsed time: {end-start}')

100%|██████████| 10/10 [02:54<00:00, 17.46s/it]

Elapsed time: 174.5880789756775


### Parallel ExIFFI

In [6]:
from sklearn.preprocessing import StandardScaler
import time
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=np.zeros(X_train.shape[0])
y_test=np.ones(X_test.shape[0])
y=np.concatenate([y_train,y_test])
X_test=np.r_[X_train,X_test]
scaler2=StandardScaler()
X=scaler2.fit_transform(X)
EDIFFI=Extended_DIFFI_parallel(300,max_depth=100,subsample_size=256,plus=1)
EDIFFI.set_num_processes(12)

dim=X.shape[1]
pwd=os.path.dirname(os.getcwd())


start=time.time()
imps,plt_data=compute_imps(EDIFFI,X,X,10,name,pwd,dim,f=6)
end=time.time()

print(f'Elapsed time: {end-start}')

100%|██████████| 10/10 [01:29<00:00,  8.92s/it]

Elapsed time: 89.17974472045898


#### Results Ionosphere

Serial -> 174.5880789756775

8 threads

Parallel -> 147.8202040195465

12 threads

Parallel -> 137.81197714805603

In [16]:
from sklearn.preprocessing import StandardScaler
import time
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
y_train=np.zeros(X_train.shape[0])
y_test=np.ones(X_test.shape[0])
y=np.concatenate([y_train,y_test])
X_test=np.r_[X_train,X_test]
scaler2=StandardScaler()
X=scaler2.fit_transform(X)
EDIFFI=Extended_DIFFI_parallel(300,max_depth=100,subsample_size=256,plus=1)

In [17]:
EDIFFI.fit(X_train)

## Diabetes Dataset

In [23]:
X,y=load_data_csv('diabetes')
X.shape,y.shape

((100000, 4), (100000,))

# Test Results 

## Parallel

In [32]:
stats=np.load('test_stat_parallel.npz',allow_pickle=True)
data_parallel=stats['importances_matrix'].tolist()
time_data_parallel=stats['execution_time_stat']

In [33]:
time_data_parallel

array({'mean': 3.36362202167511, 'std': 0.23129910849918273}, dtype=object)

In [34]:
data_parallel.keys()

dict_keys(['Execution 0', 'Execution 1', 'Execution 2', 'Execution 3', 'Execution 4', 'Execution 5', 'Execution 6', 'Execution 7', 'Execution 8', 'Execution 9'])

## Serial

In [29]:
stats=np.load('test_stat_serial.npz',allow_pickle=True)
data_serial=stats['importances_matrix'].tolist()
time_data_serial=stats['execution_time_stat']

In [30]:
time_data_serial

array({'mean': 3.5636572360992433, 'std': 0.5714168745774968},
      dtype=object)

In [31]:
data_serial.keys()

dict_keys(['Execution 0', 'Execution 1', 'Execution 2', 'Execution 3', 'Execution 4', 'Execution 5', 'Execution 6', 'Execution 7', 'Execution 8', 'Execution 9'])

Check if `data_parallel` and `data_serial` are equal


In [36]:
for k in data_serial.keys():
    print(np.sum(data_serial[k]-data_parallel[k]))

-5.995204332975845e-14
1.532107773982716e-13
-3.4638958368304884e-13
7.327471962526033e-14
5.306866057708248e-13
-3.774758283725532e-13
-3.197442310920451e-13
1.3522516439934407e-12
-1.2434497875801753e-13
-1.0769163338864018e-12


Host capri

    Hostname capri.dei.unipd.it
    User p1026u27